# Mecab ユーザー辞書作り

##### 手順

1. ユーザ辞書の準備として、CSVファイルを作成
2. コマンドラインで DIC ファイルを作成
3. 作成した辞書にパスを通す

### 1. ユーザ辞書作成の準備として、CSVファイルを作成

In [1]:
require 'csv'

true

In [2]:
original_data = {
  local_foods: 'local_foods.csv'
}

{:local_foods=>"local_foods.csv"}

In [3]:
CSV.open("localfoods_dictionary.csv", 'w') do |csv|
    original_data.each do |type, filename|
        next unless File.file? filename
        open(filename).each do |title|
            title.strip!
            
            next if title =~ %r(^[+-.$()?*/&%!"'_,]+)
            next if title =~ /^[-.0-9]+$/
            next if title =~ /曖昧さ回避/
            next if title =~ /_\(/
            next if title =~ /^PJ:/
            next if title =~ /の登場人物/
            next if title =~ /一覧/
                                                     
            title_length = title.length
                                                     
            if title_length > 3
             score = [-36000.0, -400 * (title_length ** 1.5)].max.to_i
             csv << [title, nil, nil, score, '名詞', '一般', '*', '*', '*', '*', title, '*', '*', type]
            end
        end
     end
end


{:local_foods=>"local_foods.csv"}

参考:http://qiita.com/ynakayama/items/388c82cbe14c65827769

### 2. コマンドラインで DIC ファイルを作成

コマンドラインで以下を実行する。

・/usr/local/Cellar/mecab/0.996/libexec/mecab/mecab-dict-index -d /usr/local/lib/mecab/dic/ipadic -u /Users/runa/python/All_something_in_Japan/local_foos.dic -f utf-8 -t utf-8 /Users/runa/python/All_something_in_Japan/localfoods_dictionary.csv

mecabの辞書を置く場所にコピー

・cp local_foos.dic /usr/local/lib/mecab/dic

### 3. 作成した辞書にパスを通す

・vim /usr/local/etc/mecabrc

・cat /usr/local/etc/mecabrc

;
; Configuration file of MeCab
;
; 
$Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;

dicdir =  /usr/local/lib/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

userdic_wikipedia = /usr/local/lib/mecab/dic/custom.dic

userdic_schools = /usr/local/lib/mecab/dic/schools.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n

### 確認

In [4]:
require 'uri'
require 'sqlite3'
require 'natto'
require 'mecab'

true

In [5]:
userdic_path = "./local_foos.dic"

"./local_foos.dic"

In [6]:
c = MeCab::Tagger.new("-u #{userdic_path}")

#<MeCab::Tagger:0x007fefe2369d20 @__swigtype__="_p_MeCab__Tagger">

In [7]:
print c.parse("いちご煮祭り")

いちご煮	名詞,一般,*,*,*,*,いちご煮,*,*,local_foods
祭り	名詞,一般,*,*,*,*,祭り,マツリ,マツリ
EOS
